# Contens

- 1. Deskripsi Proyek
- 2. Persiapan Data
- 3. Pelatihan Model
  - 3.1 Linear Regression 
  - 3.2 Decission Tree
  - 3.3 RandomForest
  - 3.4 LGBM
3. Analisis Model

## Deskripsi Proyek

Rusty Bargain adalah perusahaan jual beli mobil bekas yang sedang mengembangkan sebuah aplikasi untuk menarik pembeli baru. Dalam aplikasi tersebut, Anda bisa dengan cepat mengetahui nilai pasar mobil Anda. Anda memiliki akses ke data historis, spesifikasi teknis kendaraan, versi model kendaraan, dan harga kendaraan. Tugas Anda adalah membuat suatu model yang mampu menentukan nilai pasar mobil. 

Rusty Bargain tertarik pada:

- kualitas prediksi;
- kecepatan model dalam memprediksi;
- waktu yang diperlukan untuk melatih model

## Persiapan Data

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns

import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing
import lightgbm as lgb


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier


In [2]:
#muat data
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
#cek data
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
#cek ukuran data
df.shape

(354369, 16)

In [5]:
#info dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

catatan :
- dataset memiliki 354369 baris dan 16 kolom
- terdapat kolom dengan nilai tetap/konstan 0 pada kolom NumberOfPictures, segingga kolom ini tidak akan digunakan analisa lebih lanjut
- kolom yang memuat informasi tanggal seperti DateCrawled, DateCreated dan LastSeen serta kode pos akan dihilangkan karena kurang relevan untuk analisa lebih lanjut

In [6]:
#cek jumlah nilai dalam kolom price
df['Price'].value_counts() / df.shape[0]*100

0        3.039769
500      1.600027
1500     1.522142
1000     1.311909
1200     1.296389
           ...   
13180    0.000282
10879    0.000282
2683     0.000282
634      0.000282
8188     0.000282
Name: Price, Length: 3731, dtype: float64

In [7]:
#cek korelasi di kolom price
df.corr()['Price']

Price                1.000000
RegistrationYear     0.026916
Power                0.158872
Mileage             -0.333199
RegistrationMonth    0.110581
NumberOfPictures          NaN
PostalCode           0.076055
Name: Price, dtype: float64

catatan :
- nilai pasar dari mobil ada pada kolom 'Price'
- harga mobil terlihat dipengaruhi / berkorelasi positif dengan tahun registrasi, power, tahun dan bulan registrasi serta berkorelasi negatif dengan odometer kendaraan

In [8]:
#hapus kolom yang tidak perlu
df = df.drop(['DateCrawled','DateCreated','PostalCode','LastSeen'], axis=1)

In [9]:
#tampilan dataset yang baru
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,0
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,0
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,0
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,0
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,0


In [10]:
#deskripsi dataset yang baru
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0


In [11]:
#periksa kolom NumberOfPictures
df['NumberOfPictures'].value_counts()

0    354369
Name: NumberOfPictures, dtype: int64

In [12]:
#hapus kolom dengan nilai konstan
df = df.drop(['NumberOfPictures'], axis=1)

In [13]:
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


catatan :
- jumlah baris/data dengan nilai 0 pada NumberOfPictures sejumlah baris/data dari dataset = nilai NumberOfPictures adalah konstan angka 0
- selanjutnya kolom NumberOfPictures ini drop dari dataset sebelum analisa lebih lanjut


In [14]:
#periksa kolom RegistrationYear
df['RegistrationYear'].describe()

count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

In [15]:
#hapus data yang aneh pada kolom RegistrationYear dan Power
df = df[df['RegistrationYear'] < 2024]
df = df[df['RegistrationYear'] > 2000]
df = df[df['Power'] !=0]. reset_index(drop=True)

In [16]:
##periksa kolom RegistrationYear setelah dihilangkan nilai yang aneh
df['RegistrationYear'].describe()

count    206829.000000
mean       2006.938829
std           4.484838
min        2001.000000
25%        2003.000000
50%        2006.000000
75%        2010.000000
max        2019.000000
Name: RegistrationYear, dtype: float64

catatan :
- data yang aneh pada kolom RegistrationYear terlihat dari tahun min 1000 dan maximum 9999 (kemungkinan salah input data)
- jadi RegistrationYear di filter yaitu untuk mobil yang di registrasi antara tahun 2000 dan 2024
- kemudian data yang aneh selanjutnya adalah kolom power yang terisi angka 0 ( kemungkinan salah input data ), ini juga di drop

In [17]:
#cek ukuran dataset yang baru
df.shape

(206829, 11)

In [18]:
#periksa missing value
df.isnull().sum()

Price                    0
VehicleType          19414
RegistrationYear         0
Gearbox               3897
Power                    0
Model                 7100
Mileage                  0
RegistrationMonth        0
FuelType             13047
Brand                    0
NotRepaired          27212
dtype: int64

- semua missing value terjadi pada kolom dengan tipe data object, yang berisi informasi data/keterangan tambahan
- semua informasi yang hilang akan di isi dengan 'unknown'

In [19]:
#missing value di isi dengan 'unknown'
df = df.fillna('unknown')
df.isnull().sum()

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
dtype: int64

In [20]:
#encoding
categorical_features = [
    'VehicleType',
    'Gearbox',
    'Model',
    'FuelType',
    'Brand',
    'NotRepaired',
]

In [21]:
#menghitung jumlah kolom baru hasil encoding data categorical
for feature in categorical_features :
    print(f'features {feature} {len(df[feature].value_counts())}')

features VehicleType 9
features Gearbox 3
features Model 246
features FuelType 8
features Brand 40
features NotRepaired 3


- data Model dan Brand menyumbang hasil encoding yang sangat besar yaitu 246 dan 40 kolom
- supaya efektif kita akan kesampingkan data hasil encoding dari model dan brand

In [22]:
#hapus kolom Model dan Brand hasil encoding
df_ohe = df.drop(['Model','Brand'], axis = 1)
df_ohe = pd.get_dummies(df_ohe)

In [23]:
#cek ukuran dataset hasil encoding
df_ohe.shape

(206829, 28)

In [24]:
#cek dataset hasil encoding
df_ohe.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,FuelType_unknown,NotRepaired_no,NotRepaired_unknown,NotRepaired_yes
0,18300,2011,190,125000,5,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
1,9800,2004,163,125000,8,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,1500,2001,75,150000,6,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,3600,2008,69,90000,7,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,2200,2004,109,150000,8,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


catatan :
- encoding data dilakukan pada kolom :
  - features VehicleType 9
  - features Gearbox 3
  - features FuelType 8
  - features NotRepaired 3
- sehingga ukuran dataset yang baru menjadi 206829 baris 28 kolomm

In [25]:
#splitting dataset
df_ohe_train_valid, df_ohe_test = train_test_split(df_ohe,test_size=0.15, random_state=12345)
df_ohe_train,df_ohe_valid = train_test_split(df_ohe_train_valid, test_size=0.25, random_state=54321)

print (df_ohe_train.shape)
print (df_ohe_valid.shape)
print (df_ohe_test.shape)

(131853, 28)
(43951, 28)
(31025, 28)


In [26]:
#cek korelasi pada kolom Price
df_ohe.corr(method = 'spearman')['Price'].sort_values()

Mileage                   -0.357856
VehicleType_unknown       -0.252298
VehicleType_small         -0.244130
Gearbox_manual            -0.222709
NotRepaired_yes           -0.219697
NotRepaired_unknown       -0.176673
FuelType_unknown          -0.169453
FuelType_petrol           -0.140013
Gearbox_unknown           -0.083580
VehicleType_other         -0.016286
FuelType_lpg              -0.016240
FuelType_cng              -0.008564
FuelType_other            -0.004266
FuelType_electric          0.009594
FuelType_hybrid            0.025984
VehicleType_bus            0.039304
VehicleType_wagon          0.060886
RegistrationMonth          0.086114
VehicleType_coupe          0.096151
VehicleType_convertible    0.124538
VehicleType_sedan          0.129130
VehicleType_suv            0.176950
FuelType_gasoline          0.230988
Gearbox_auto               0.256892
NotRepaired_no             0.291170
RegistrationYear           0.438915
Power                      0.525341
Price                      1

catatan :
- dataset di pisah secara acak sejumlah 15% untuk testing, 25% untuk validasi dan sisanya untuk proses training
- 2 parameter yang berkorelasi paling besar terhadap harga kendaraan adalah besarnya kapasitas mesin dan tahun registrasi kendaraan 

## Pelatihan Model

In [27]:
# fungsi untuk menghitung rmse
def rmse (target, prediction):
    return mean_squared_error (target,prediction)**0.5

In [28]:
#penyusunan features dan target untuk pelatihan model
features_train = df_ohe_train.drop(['Price'], axis = 1)
target_train = df_ohe_train['Price']

features_valid = df_ohe_valid.drop(['Price'], axis = 1)
target_valid = df_ohe_valid['Price']

features_test = df_ohe_test.drop(['Price'], axis = 1)
target_test = df_ohe_test['Price']


catatan :
- penyusunan features dan target untuk pelatihan model dilakukan pada kolom 'Price' sebagai representasi Nilai pasar kendaraan

### Linear Regresion

- Linear Regression sebagai acuan / baseline / sanity check

In [29]:
%%time

model = LinearRegression()
model.fit(features_train, target_train)


CPU times: user 156 ms, sys: 104 ms, total: 260 ms
Wall time: 247 ms


LinearRegression()

In [30]:
%%time

pred_train = model.predict (features_train)
pred_valid = model.predict (features_valid)
pred_test = model.predict (features_test)

print('Train RMSE :', rmse(target_train,pred_train).round(5))
print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))
print('Test RMSE :', rmse(target_test,pred_test).round(5))

Train RMSE : 3122.71756
Valid RMSE : 3107.08173
Test RMSE : 3104.9996
CPU times: user 40.7 ms, sys: 56.2 ms, total: 96.9 ms
Wall time: 35.7 ms


### Decission Tree

In [31]:
%%time
for depth in [1,2,3,4,6,None]:
    model = DecisionTreeRegressor(max_depth=depth)
    model.fit(features_train,target_train)
    
    pred_train = model.predict(features_train)
    pred_valid = model.predict(features_valid)
    
    print('Depth :', depth)
    print('Train RMSE :', rmse(target_train,pred_train).round(5))
    print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))

Depth : 1
Train RMSE : 4228.0715
Valid RMSE : 4236.05785
Depth : 2
Train RMSE : 3672.40154
Valid RMSE : 3668.40637
Depth : 3
Train RMSE : 3339.30118
Valid RMSE : 3343.69326
Depth : 4
Train RMSE : 2990.46244
Valid RMSE : 2996.03837
Depth : 6
Train RMSE : 2575.35099
Valid RMSE : 2584.37382
Depth : None
Train RMSE : 860.20808
Valid RMSE : 2570.30993
CPU times: user 1.85 s, sys: 2.94 ms, total: 1.85 s
Wall time: 1.85 s


In [32]:
%%time

pred_train = model.predict (features_train)
pred_valid = model.predict (features_valid)
pred_test = model.predict (features_test)

print('Train RMSE :', rmse(target_train,pred_train).round(5))
print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))
print('Test RMSE :', rmse(target_test,pred_test).round(5))

Train RMSE : 860.20808
Valid RMSE : 2570.30993
Test RMSE : 2574.06801
CPU times: user 107 ms, sys: 3.22 ms, total: 110 ms
Wall time: 107 ms


### Random Forest

In [33]:
for depth in [1,2,3,4,6,8, None]:
    model = RandomForestRegressor(max_depth=depth, n_estimators = 100)
    model.fit(features_train,target_train)
    
    pred_train = model.predict(features_train)
    pred_valid = model.predict(features_valid)
    
    print('Depth :', depth)
    print('Train RMSE :', rmse(target_train,pred_train).round(5))
    print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))

Depth : 1
Train RMSE : 4228.07197
Valid RMSE : 4236.05529
Depth : 2
Train RMSE : 3671.11015
Valid RMSE : 3667.21113
Depth : 3
Train RMSE : 3310.98869
Valid RMSE : 3314.57387
Depth : 4
Train RMSE : 2949.84802
Valid RMSE : 2954.15377
Depth : 6
Train RMSE : 2526.44319
Valid RMSE : 2533.70457
Depth : 8
Train RMSE : 2306.2748
Valid RMSE : 2344.34129
Depth : None
Train RMSE : 1073.13249
Valid RMSE : 2078.16557


In [34]:
%%time

model = RandomForestRegressor(max_depth=8, n_estimators = 100)
model.fit(features_train,target_train)

CPU times: user 24.6 s, sys: 63.9 ms, total: 24.6 s
Wall time: 24.6 s


RandomForestRegressor(max_depth=8)

In [35]:
%%time

pred_train = model.predict (features_train)
pred_valid = model.predict (features_valid)
pred_test = model.predict (features_test)

print('Train RMSE :', rmse(target_train,pred_train).round(5))
print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))
print('Test RMSE :', rmse(target_test,pred_test).round(5))

Train RMSE : 2304.09874
Valid RMSE : 2341.01006
Test RMSE : 2338.68562
CPU times: user 1.93 s, sys: 0 ns, total: 1.93 s
Wall time: 1.94 s


### LGBM

In [36]:
%%time

model = lgb.LGBMRegressor(num_iterations=20, verbose=0, metric='rmse')
model.fit(features_train, target_train, 
          eval_set=(features_valid, target_valid))



/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's rmse: 4446.61
[2]	valid_0's rmse: 4163
[3]	valid_0's rmse: 3916.85
[4]	valid_0's rmse: 3696.55
[5]	valid_0's rmse: 3509.78
[6]	valid_0's rmse: 3343.06
[7]	valid_0's rmse: 3198.98
[8]	valid_0's rmse: 3074.64
[9]	valid_0's rmse: 2967.22
[10]	valid_0's rmse: 2874.61
[11]	valid_0's rmse: 2793.9
[12]	valid_0's rmse: 2723.73
[13]	valid_0's rmse: 2660.88
[14]	valid_0's rmse: 2607.48
[15]	valid_0's rmse: 2560.3
[16]	valid_0's rmse: 2518.09
[17]	valid_0's rmse: 2481.54
[18]	valid_0's rmse: 2447.05
[19]	valid_0's rmse: 2419.97
[20]	valid_0's rmse: 2394.84
CPU times: user 1.86 s, sys: 3.41 ms, total: 1.87 s
Wall time: 1.88 s


LGBMRegressor(metric='rmse', num_iterations=20, verbose=0)

In [37]:
%%time
pred_train = model.predict (features_train)
pred_valid = model.predict (features_valid)
pred_test = model.predict (features_test)

print('Train RMSE :', rmse(target_train,pred_train).round(5))
print('Valid RMSE :', rmse(target_valid,pred_valid).round(5))
print('Test RMSE :', rmse(target_test,pred_test).round(5))

Train RMSE : 2382.51669
Valid RMSE : 2394.83998
Test RMSE : 2390.46661
CPU times: user 424 ms, sys: 0 ns, total: 424 ms
Wall time: 403 ms


## Analisis Model

Hasil kinerja model:
- Linear Regression
  - Training model
    - CPU times: user 205 ms, sys: 45.6 ms, total: 251 ms
    - Wall time: 239 ms
  - Prediksi model
    - Train RMSE : 3122.71756
    - Valid RMSE : 3107.08173
    - Test RMSE : 3104.9996
    - CPU times: user 25.8 ms, sys: 57.5 ms, total: 83.3 ms
    - Wall time: 40.6 ms
- Decission Tree
  - Training model
    - CPU times: user 1.96 s, sys: 0 ns, total: 1.96 s
    - Wall time: 1.97 s
  - Prediksi model  
    - Train RMSE : 860.20808
    - Valid RMSE : 2567.09035
    - Test RMSE : 2590.21463
    - CPU times: user 118 ms, sys: 0 ns, total: 118 ms
    - Wall time: 115 ms
- RandomForrest
  - Training model
    - CPU times: user 26.6 s, sys: 87.8 ms, total: 26.7 s
    - Wall time: 26.7 s
  - Prediksi model
    - Train RMSE : 2306.24578
    - Valid RMSE : 2342.71906
    - Test RMSE : 2341.61124
    - CPU times: user 1.98 s, sys: 0 ns, total: 1.98 s
    - Wall time: 1.99 s

- LGBM
  - Training model
    - CPU times: user 1.14 s, sys: 11.4 ms, total: 1.15 s
    - Wall time: 1.16 s
  - Prediksi model
    - Train RMSE : 2382.51669
    - Valid RMSE : 2394.83998
    - Test RMSE : 2390.46661
    - CPU times: user 461 ms, sys: 3.87 ms, total: 465 ms
    - Wall time: 432 ms
  

Kesimpulan :

- kualitas prediksi terbaik adalah menggunakan RandomForrest
- kecepatan model dalam memprediksi menggunakan Linear Regression
- waktu yang diperlukan untuk melatih model paling pendek adalah menggunakan Linear Regression

Rekomendasi untuk Rusty Bargain :

kombinasi untuk ketepatan prediksi dan lama waktu prediksi terbaik adalah menggunakan model Decission tree

